In [1]:
import os
import pandas as pd
from collections import defaultdict

In [2]:
LOCALDIR = "/home/tfaraut/mnt/genologin/dynawork/CircRNA/"

def read_sj_tab_out(inputfile, sample, sample_unit, read, min_reads=10):
    data = []
    with open(inputfile) as fin:
        for line in fin:
            fields = line.rstrip().split()
            if int(fields[6]) >  min_reads:
                key = "%s:%s-%s:%s" % (fields[0], fields[1], fields[2], fields[3])
                data.append([ key, sample_unit, read, fields[6]])
    df = pd.DataFrame(data, columns=['junction', 'sample_unit', 'read', 'nb'])
    df['nb'] = pd.to_numeric(df['nb'])
    return df

def strand_str(strand):
    return "+" if strand == "1" else "-"


# The junction here keep the information of acceptor and donor site
# according to STAR documentation 
def junctionkeys(fields):
    strand = strand_str(fields[3])
    if strand == "+": 
        key_donor = "%s:%s:%s:d" % (fields[0], fields[1], strand)
        key_acceptor = "%s:%s:%s:a" % (fields[0], fields[2], strand)
    else:
        key_donor = "%s:%s:%s:d" % (fields[0], fields[2], strand)
        key_acceptor = "%s:%s:%s:a" % (fields[0], fields[1], strand)
    return key_donor, key_acceptor


def read_sj_tab_out_singlejunction(inputfile, sample, sample_unit, read, min_reads=10):
    data = []
    with open(inputfile) as fin:
        for line in fin:
            fields = line.rstrip().split()
            if int(fields[6]) >  min_reads:
                key_donor, key_acceptor = junctionkeys(fields)
                data.append([ key_donor, sample_unit, read, fields[6]])
                data.append([ key_acceptor, sample_unit, read, fields[6]])
    df = pd.DataFrame(data, columns=['junction', 'sample_unit', 'read', 'nb'])
    df['nb'] = pd.to_numeric(df['nb'])
    return df

def get_sj_reads(sample, sample_unit, mapdir):
    lmapdir = mapdir.replace("/work2/genphyse/dynagen/tfaraut/CircRNA/", LOCALDIR)
    dfs = []
    for read in ['R1', 'R2']:
        sj_file = os.path.join(lmapdir, "se", read, "SJ.out.tab")
        dfs.append(read_sj_tab_out(sj_file, sample, sample_unit, read))   
    df = pd.concat(dfs)
    df_group = df.groupby(['junction']).aggregate({'nb': 'sum', 'sample_unit': 'first'})
    return df_group

def get_sj_reads_singlejuntion(sample, sample_unit, mapdir):
    lmapdir = mapdir.replace("/work2/genphyse/dynagen/tfaraut/CircRNA/", LOCALDIR)
    dfs = []
    for read in ['R1', 'R2']:
        sj_file = os.path.join(lmapdir, "se", read, "SJ.out.tab")
        dfs.append(read_sj_tab_out_singlejunction(sj_file, sample, sample_unit, read))   
    df = pd.concat(dfs)
    df_group = df.groupby(['junction']).aggregate({'nb': 'sum', 'sample_unit': 'first'})
    return df_group

In [3]:
medata_file = "samples_cow_testis_neonat.tsv"
metadata = pd.read_csv(medata_file, sep="\t").set_index('sample_unit')
metadata.head()

,sample,species,mapdir
sample_unit,,,
SRR7175531,cow-testis-neonat1,cow,/work2/genphyse/dynagen/tfaraut/CircRNA/mappin...
SRR7175526,cow-testis-neonat2,cow,/work2/genphyse/dynagen/tfaraut/CircRNA/mappin...
SRR7175527,cow-testis-neonat3,cow,/work2/genphyse/dynagen/tfaraut/CircRNA/mappin...


In [4]:
df = defaultdict(list)
for index, row in metadata.iterrows():
    sample = row['sample']
    print("Reading %s %s " % (sample, index))
    df[sample].append(get_sj_reads_singlejuntion(sample, index, row['mapdir']))

Reading cow-testis-neonat1 SRR7175531 
Reading cow-testis-neonat2 SRR7175526 
Reading cow-testis-neonat3 SRR7175527 


In [5]:
df_samples = []
for sample in df:
    df_new = pd.concat(df[sample]).groupby(level=0).aggregate({'nb': 'sum'})
    df_new.columns = [sample]
    df_samples.append(df_new)

In [6]:
df_full = df_samples[0].join(df_samples[1:]).fillna(0)
df_full = df_full.rename(columns={col: col+"-linear" for col in df_full.columns})
df_full = df_full[df_full.min(axis=1)>10]

In [7]:
df_full.head()

,cow-testis-neonat1-linear,cow-testis-neonat2-linear,cow-testis-neonat3-linear
junction,,,
10:1000040:-:a,88.0,141.0,139.0
10:100119603:+:d,68.0,112.0,88.0
10:100125353:+:a,68.0,112.0,88.0
10:100125429:+:d,98.0,121.0,127.0
10:100127500:+:a,98.0,109.0,127.0


In [8]:
df_full.to_csv("forwardcounts_cow_testis_neonat.tsv", sep="\t", index=True)